# Ноутбук для демонстрации `Composer` - объединения детектора и классификатора.

In [ ]:
from datetime import datetime
now = datetime.now
import random
import os

import matplotlib.pyplot as plt
import numpy as np
import pathlib
import shutil
import cv2
import torch

# Зафиксируем состояние случайных чисел
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)
random.seed(RANDOM_STATE)
%matplotlib inline
plt.rcParams["figure.figsize"] = (17,10)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')    
device

In [ ]:
try:
    PROJECT_ROOT = pathlib.Path(os.readlink(f'/proc/{os.environ["JPY_PARENT_PID"]}/cwd'))
except FileNotFoundError:
    __file = %pwd
    PROJECT_ROOT = pathlib.Path(__file).parents[1]

DATA_DIR = PROJECT_ROOT / 'SignDetectorAndClassifier' / 'data'
VIDEO_DIR = DATA_DIR / 'reg_videos'

video_path = str(VIDEO_DIR / '2.mp4')
print(video_path)

In [ ]:
from maddrive_adas.sign_det.composer import BasicSignsDetectorAndClassifier
from maddrive_adas.sign_det.base import (
    AbstractComposer, DetectedInstance,
    AbstractSignClassifier, AbstractSignDetector
)
from maddrive_adas.sign_det.classifier import EncoderBasedClassifier
from maddrive_adas.sign_det.detector import YoloV5Detector

# FIX THIS PATH BY YOURS
DETECTOR_ARCHIVE = PROJECT_ROOT / 'maddrive_adas' / 'sign_det' / 'detector_config_img_size'
CLASSIFIER_ARCHIVE = PROJECT_ROOT / 'SignDetectorAndClassifier' / 'data' / 'saved_model'

c: AbstractSignClassifier = EncoderBasedClassifier(
    config_path=str(CLASSIFIER_ARCHIVE)
)

d: AbstractSignDetector = YoloV5Detector(
    config_path=str(DETECTOR_ARCHIVE)
)

composer: AbstractComposer = BasicSignsDetectorAndClassifier(
    classifier=c,
    detector=d
)

In [ ]:
from IPython.display import display, Image

video = cv2.VideoCapture(video_path)
display_handle1=display(1, display_id=True)

# TODO: plot detector confidences

# const for plot
COLOR = (0, 255, 0)
DELTA = 15

while True:
    # save initial time
    t0 = now()
    
    # get video frame
    _, frame_src = video.read()
    
    # conver BRG to RGB for model input and get composer predition
    frame_src = cv2.cvtColor(frame_src, cv2.COLOR_BGR2RGB)    
    detected_instance, predicted_signs = composer.detect_and_classify(frame_src)
    
    for idx, sign in enumerate(predicted_signs):
        COORD_ARR, conf = detected_instance.get_abs_roi(idx)
        frame_src = cv2.rectangle(
            frame_src, 
            (COORD_ARR[0], COORD_ARR[1]), 
            (COORD_ARR[2], COORD_ARR[3]), 
            COLOR, 
            3
        )
        
        frame_src = cv2.putText(frame_src, sign[0], 
                           (COORD_ARR[0], COORD_ARR[3] + DELTA),
                           cv2.FONT_HERSHEY_SIMPLEX,
                           0.7, (255, 255, 255),
                           3, cv2.LINE_AA
                          )
        
        frame_src = cv2.putText(frame_src, sign[0],
                           (COORD_ARR[0], COORD_ARR[3] + DELTA),
                           cv2.FONT_HERSHEY_SIMPLEX,
                           0.7, (0, 0, 0),
                           1, cv2.LINE_AA
                          )
        
    dt = now() - t0
    frame_src = cv2.putText(
        frame_src, 
        'fps:' + str(round(1 / dt.total_seconds() , 2)),
        (0, 70),
        cv2.FONT_HERSHEY_SIMPLEX,
        3, (0, 0, 0),
        3, cv2.LINE_AA
    )
    
    # transform back to BGR for cv2
    frame_src = cv2.cvtColor(frame_src, cv2.COLOR_RGB2BGR)
    _, frame_src = cv2.imencode('.jpeg', frame_src)
        
    display_handle1.update(Image(data=frame_src.tobytes()))